## XGBデフォルト

In [1]:
import numpy as np
import pandas as pd

import xgboost
from xgboost import XGBClassifier
from sklearn.metrics import mean_squared_error, confusion_matrix 
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split


In [2]:
# pickle ファイルから読み込み
train_pkl = pd.read_pickle('./train.pk1')

In [3]:
train_pkl.shape

(242150, 10)

## 訓練データとテストデータに分割

In [4]:
# object 型削除
train_pkl.drop(['grade', 'purpose'], axis=1, inplace=True)

In [5]:
# 訓練とテストデータに分割
train, test = train_test_split(train_pkl, test_size=0.2, random_state=42)

In [6]:
# ターゲットと特徴量の分割
train_X = train.iloc[:, 1:-1]
train_y = train.loan_status

## 訓練データで訓練

In [13]:
train_X.shape, train_y.shape

((193720, 6), (193720,))

In [7]:
XGB = XGBClassifier()

In [8]:
XGB = XGB.fit(train_X, train_y)

In [9]:
XGB.feature_importances_

array([0.04594566, 0.15356599, 0.63823044, 0.08324791, 0.05168985,
       0.02732016], dtype=float32)

In [10]:
sorted(
    zip(map(lambda x: round(x, 3), XGB.feature_importances_), train_X.columns),
    reverse=True)

[(0.638, 'interest_rate'),
 (0.154, 'term'),
 (0.083, 'employment_length'),
 (0.052, 'credit_score'),
 (0.046, 'loan_amnt'),
 (0.027, 'application_type')]

## テストデータで実行

In [15]:
# ターゲットと特徴量の分割
test_x = test.iloc[:, 1:-1]
test_y = test.loan_status

In [16]:
test_x.shape, test_y.shape, test.shape

((48430, 6), (48430,), (48430, 8))

In [17]:
pred_y = XGB.predict(test_x)

In [18]:
confusion_matrix(test_y, pred_y)

array([[39465,   435],
       [ 7870,   660]], dtype=int64)

In [23]:
test_y.sum(), pred_y.sum()

(8530, 1095)

In [19]:
accuracy_score(test_y, pred_y)

0.8285153830270493

In [20]:
precision_score(test_y, pred_y)

0.6027397260273972

In [21]:
recall_score(test_y, pred_y)

0.07737397420867527

In [22]:
f1_score(test_y, pred_y)

0.13714285714285715

## 検証データで実行

In [24]:
# 検証データ読み込み
valid = pd.read_pickle('./test.pk1')

In [25]:
valid.shape

(26900, 9)

In [28]:
# ID の保存
valid_pass = valid.id.values

In [32]:
# valid_X = valid.iloc[:, 1:]
valid_X = valid.copy()

In [33]:
valid_X.drop(['id', 'grade', 'purpose'], axis=1, inplace=True)

In [34]:
valid_X.shape, train_X.shape

((26900, 6), (193720, 6))

In [35]:
pred_valid_y = XGB.predict(valid_X)

In [36]:
pred_valid_y.shape

(26900,)

In [ ]:
# type(valid_pass), type(pred_valid_y)

In [38]:
result_df = pd.DataFrame(pred_valid_y, valid_pass, columns=['y'])

In [39]:
result_df.to_csv("./XGB_1.csv", header=False)